# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from fbprophet import Prophet
import plotly
plotly.tools.set_credentials_file(username='partha1989', api_key='Y2Yzydnp97ZO3Qoi8f1y')
import plotly.plotly as py
import plotly.graph_objs as go
from fbprophet.diagnostics import cross_validation, performance_metrics
import pickle
from sklearn.metrics import mean_squared_error
from math import sqrt
from geolib import geohash
from multiprocessing import Pool, cpu_count

# Function declaration

In [2]:
# ***Function to create line chart***
def plotlyline(x_val,y_val,types,name):
    line = go.Scatter(x = x_val,y = y_val,mode = types,name = name)
    return line

# ***Function to create Bar chart***
def plotlybar(x_val,y_val):
    bar = go.Bar(x=x_val,y=y_val,text=y_val,textposition = 'auto',name='bars')
    return bar

# ***Function to create Geo map***
def plotlygeo(lat,long):
    data = [go.Scattergeo(lat = lat ,lon = long)]
    return data

# ***Function for Box plot***
def plotlybox(y):
    data = [go.Box(y=y,boxpoints='all',jitter=0.3,pointpos=-1.8)]
    return data

# ***Function to create dummy dates and assign Index to it***
def dummydate(startdate, index_end , index_start):
    date_list = pd.DataFrame()
    date_list["Date"] = [(startdate + timedelta(days=x)).strftime("%Y-%m-%d") for x in range(0, index_end)]
    date_list["Date"] =pd.to_datetime(date_list.Date, format='%Y-%m-%d').dt.date
    date_list = date_list.sort_values(by='Date')
    date_list['day'] = np.arange(date_list.shape[0]) + index_start
    return date_list

def dataprep(startdate, index, dataframe):
    ## Dummy start date for forecasting - Replace with actual dates
    startdate = datetime.strptime(startdate, '%Y-%m-%d')
    ## Creating dummy date with index
    datelist = dummydate(startdate, index, dataframe["day"].min() )
    dump_fin = pd.merge(dataframe, datelist, how='inner')
    dump_fin["Datetime"] = dump_fin["Date"].astype(str)+ " " + dump_fin["timestamp"].astype(str)
    dump_fin["Datetime"] = pd.to_datetime(dump_fin["Datetime"])
    ## Creating dataset with required columns
    demand = dump_fin.filter(['geohash6','Datetime','demand'], axis=1)
    demand["Datetime"] = pd.to_datetime(demand["Datetime"])
    return demand

# ***Function to convert geohash to Lat & Long***
def geohashlatlong(dataframe):
    geohash6 = dataframe["geohash6"].unique()
    geo_data = pd.DataFrame()
    for geo in geohash6:
        lat = geohash.decode(geo)[0]
        long = geohash.decode(geo)[1]
        temp = pd.DataFrame({'Geo': [geo],'lat': [lat], 'long': [long]})
        geo_data = geo_data.append(temp)
    return geo_data

# ***Create dataframe for geohash (by precision)***
def geohash5fn(dataframe,precision):
    geohash_5 = dataframe.copy()
    geohash_5["geohash5"] = dataframe["geohash6"].str[:precision]
    geohash_5 = geohash_5.groupby(['Datetime','geohash5'])['demand'].sum().reset_index()
    return geohash_5


# ***Create prediction and save model***
def predictiongeo(train,test,remove_geohash,export_model):
    
    daystopredict = (test["Datetime"].dt.date.max() - train["Datetime"].dt.date.max()).days
    period = round(((24 *60)/15)* int(daystopredict+6))
    train= train[~train["geohash6"].isin (remove_geohash)]
    train["hour"] =  train["Datetime"].dt.hour
    df = train.copy()
    df = df.rename(index=str, columns={"Datetime": "ds", "demand": "y"})
    #df["on_season"] = np.where(((df['ds'] >  (df["ds"].min() + timedelta(days=31)).strftime("%Y-%m-%d")) &(df['ds'] <  (df["ds"].min() + timedelta(days=58)).strftime("%Y-%m-%d")) ),1,0)
    #df["cap"] = maxi
    #df["floor"] = mini
    #hash5 = ('qp02yc', 'qp02yf', 'qp02yv', 'qp02yy', 'qp02yz')
    hash5 = df.geohash6.unique()
    
    test_val = test[test["geohash6"].isin(hash5)].copy()
    test_val = test_val.rename(index=str, columns={"Datetime": "ds", "demand": "y"})
    test_val["hour"] = test_val["ds"].dt.hour
    max_test_ds = test_val["ds"].max()
    #test_val["on_season"]=1
    fsct_test = pd.DataFrame()


    fsct_fin = pd.DataFrame()
    # Prediction by Geohash
    for geo in hash5:
        geo_df = df[df['geohash6'] == geo].copy()
        geo_test = test_val[test_val['geohash6'] == geo].copy()
        my_model = Prophet(interval_width=0.95,yearly_seasonality=False)
        my_model.add_regressor('hour')
#         my_model.add_regressor('sum_neighbours')
#         my_model.add_regressor('mean_neighbours')
#         my_model.add_regressor('min_neighbours')
#         my_model.add_regressor('max_neighbours')
#         my_model.add_regressor('std_neighbours')
        my_model.fit(geo_df)
        future_dates = my_model.make_future_dataframe( periods = period, freq = '15min' )
        future_dates["hour"] =  future_dates["ds"].dt.hour
        #future_dates = future_dates[future_dates["ds"] > max_test_ds]
        #future_dates["on_season"] = 1
        #future_dates["cap"] = maxi
        #future_dates["floor"] = mini
        forecast = my_model.predict(future_dates)
        forecast["geohash"] = geo
        fsct_fin = fsct_fin.append(forecast)
        
        if (geo_test.shape[0] > 0):
            forecast_test = my_model.predict(geo_test)
            forecast_test["geohash"] = geo
            fsct_test = fsct_test.append(forecast_test)

        # Saving model output to the file in the current working directory (cannot save in DB) and resue if needed
        if (export_model == True):
            pkl_filename = geo +"_model.pkl"
            with open(pkl_filename, 'wb') as file:  
                pickle.dump(my_model, file)
    return fsct_fin , fsct_test

# ***Replace dataframe with negative values to 0***
def replacenegative(dataframe):
    num = dataframe._get_numeric_data()
    num[num < 0] = 0
    return dataframe

def rmse(actual,predicted):
    rms = sqrt(mean_squared_error(actual, predicted))
    return rms

def predictingtest(train,test,remove_geohash):
    train= train[~train["geohash6"].isin (remove_geohash)]
    geohash = train["geohash6"].unique()
    test_val = test[test["geohash6"].isin(geohash)].copy()
    test_val = test_val.rename(index=str, columns={"Datetime": "ds", "demand": "y"})
    test_val["hour"] = test_val["ds"].dt.hour
    #test_val["on_season"]=1
    hash_val  = test_val["geohash6"].unique()
    fsct_fin = pd.DataFrame()
    for geo in hash_val:
        geo_df = test_val[test_val['geohash6'] == geo]
        pkl_filename = geo +"_model.pkl"
        with open(pkl_filename, 'rb') as fin:
            m2 = pickle.load(fin)
        fcst = m2.predict(geo_df)
        fcst["geohash"] = geo
        fsct_fin = fsct_fin.append(fcst)
    return fsct_fin

def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def createcsv(dataframe,filter_days_start,filter_days_end,filename):
    dataframe[(dataframe["day"] >= int(filter_days_start)) & (dataframe["day"] <= int(filter_days_end))].to_csv(filename,index = None, header=True)
    
def readcsv():
    data = pd.read_csv("training_final.csv")
    return data

def fillnan(dataframe, column):
    dataframe[column] = dataframe[column].fillna(0)
    return dataframe

def geohashneighbor(geohash6):
    neighbour_df = pd.DataFrame()
    for geo in geohash6:
        neighbour_dict = pd.DataFrame()
        neighbour = geohash.neighbours(geo)
        neighbour_dict["neighbour"] = list(neighbour)
        neighbour_dict["parent"] = geo
        neighbour_df = neighbour_df.append(neighbour_dict)
    return neighbour_df

def geohashneighbours(dataframe):
    neighbour =  geohashneighbor(dataframe["geohash6"].unique())
    geohash = dataframe["geohash6"].unique()
    dump = pd.DataFrame()
    for geo in geohash:
        temp_dataframe = dataframe[dataframe["geohash6"] == geo].copy()
        temp_neighbour = neighbour[neighbour["parent"] == geo]["neighbour"]
        temp_data = dataframe[dataframe["geohash6"].isin (temp_neighbour)].copy()
        neighbour_stats =  temp_data.groupby(temp_data["Datetime"]).agg({'demand' :[sum,np.mean,min,max,np.std]}).reset_index()
        neighbour_stats.columns = ["Datetime","sum_neighbours","mean_neighbours","min_neighbours","max_neighbours","std_neighbours"]
        temp_dump = pd.merge(temp_dataframe, neighbour_stats, how='left')
        dump = dump.append(temp_dump)
    return dump
        

# Importing data

In [3]:
# *** Assuming that training and test file has same column names. Day is continuous value from Train file (any value above 61) .
# Prophet assumes 0 for missing dates ***

# *** I have created Train & Test csv file to test  my code ***

# dummy = readcsv()
# createcsv(dummy ,1,56,"training.csv")
# createcsv(dummy ,57,61,"test.csv")

# *** Load training file***
dump_train = pd.read_csv("training.csv")

# *** Load test file - replace the proper file name***

dump_test = pd.read_csv("test.csv")



# Data Preparation

## Time Series forecasting

### Time series requires date for forecasting. So I have used dummy dates to extract seasonality & trend. I would request grab team to subsititute exact start date


In [4]:
# ***Enter start date for forecasting - Replace with actual dates (Please enter start date for training dataset)***

start_date = '2019-04-01'

train = dataprep( start_date, dump_train["day"].max(),dump_train)

# *** You can add exact start date of test data set ***

start_date_test =  '2019-05-18'

test = dataprep( start_date_test, ((dump_test["day"].max() - dump_test["day"].min()).astype(int))+1, dump_test)

# *** Continue running below code *** #


### Extracting features from nearby neighbours - Not used

In [5]:

# train = geohashneighbours(train)
# test = geohashneighbours(test)
# # ***Check null values***
# print(train.isnull().sum().sum())
# print(test.isnull().sum().sum())
# # *** Replace Null with 0 ***
# train = train.fillna(0)
# test = test.fillna(0)
# # ***Check null values***

In [6]:
print(train.isnull().sum().sum())
print(test.isnull().sum().sum())

0
0


# Data Visualization

In [7]:
plot_demand = dump_train.groupby('day')['demand'].sum().reset_index()
layout = dict(title = 'Box Plot Demand',
               xaxis = dict(title = 'Demand')
              )
fig = dict(data = plotlybox(pd.Series(plot_demand["demand"])), layout=layout)
py.iplot(fig, filename='Demand -boxplot')



In [8]:
day_demand = dump_train.groupby('day')['demand'].sum().reset_index()
layout = dict(title = 'Daywise Demand',
              xaxis = dict(title = 'Day'),
              yaxis = dict(title = 'Demand'),
              )
fig = dict(data = [plotlyline(day_demand["day"],day_demand["demand"],"lines","demand")], layout=layout)
py.iplot(fig, filename='Demand - Day wise')


In [9]:
geohash_demand = train.groupby(train["Datetime"].dt.hour)['demand'].sum().reset_index()
layout = dict(title = 'Geohash Demand',
              xaxis = dict(title = 'Hour'),
              yaxis = dict(title = 'Demand'),
              )
fig = dict(data = [plotlybar(geohash_demand["Datetime"],round(geohash_demand["demand"]))], layout=layout)
py.iplot(fig, filename='Demand - Hour wise')


### Seems bit fishy to have less demand on Friday where it usually peaks (from my experience in Careem - Last working day of week. By giving actual datesthings might change)

In [10]:
sorter = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
sorterIndex = dict(zip(sorter,range(len(sorter))))
daywise_demand = train.groupby(train["Datetime"].dt.strftime("%A"))['demand'].sum().reset_index()

daywise_demand['Day_id'] = daywise_demand.index

daywise_demand['Day_id_rnk'] = daywise_demand['Datetime'].map(sorterIndex)

daywise_demand.sort_values('Day_id_rnk', inplace=True)


layout = dict(title = 'Geohash Demand',
              xaxis = dict(title = 'Day Name'),
              yaxis = dict(title = 'Demand'),
              )

fig = dict(data = [plotlybar(daywise_demand["Datetime"],round(daywise_demand["demand"]))], layout=layout)
py.iplot(fig, filename='Demand - Day wise')


#### Converting Geo hash to lat and long

In [11]:
geo_data = geohashlatlong(train)

### After Plotting Lat and Long on map, all the location given in dataset lies in Indian ocean :)

In [12]:
layout = dict(
        title = 'Latitude & Longititude Maps', 
        geo = dict(
            scope='asia',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5        
        ),
    )

fig = go.Figure(data=plotlygeo(geo_data["lat"],geo_data["long"]),layout=layout )
py.iplot(fig, filename='Latitude & Longitiude')

In [13]:
# ***data is already normalized so outliers are handled***
train["demand"].describe()

count    3.845621e+06
mean     1.042759e-01
std      1.583243e-01
min      3.092217e-09
25%      1.855817e-02
50%      5.006287e-02
75%      1.198046e-01
max      1.000000e+00
Name: demand, dtype: float64

# Forecasting

### Prophet package handles missing timeseries effectively so I am not imputing it. 

### We are removing Geohash with less than 2 data points as Prophet needs min 2 or more values to predict

In [14]:
chk = train.groupby(["geohash6"])['demand'].count()
chk = chk.to_frame()
print("% of Geohash with Data points less than 3 data points  is",(chk[chk["demand"] <=2].count()/chk.shape[0])*100)
remove_geohash = chk[chk["demand"] <=2].index


% of Geohash with Data points less than 3 data points  is demand    1.430723
dtype: float64


In [15]:
# ***Prediction Removing geohash less than 2 data points for prediction. Ignoring Geohash which is not part of training set
# predictiongeo(training dataset, test dataset ,remove geohash less than 2 data points , whether to save model in
#pickle file ) ***
# Approx 3 hrs to run #


fsct_train_mdl,fsct_test_mdl =  predictiongeo(train,test,remove_geohash,False) 

/Users/parthasarathysridharan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/parthasarathysridharan/anaconda3/lib/python3.6/site-packages/pystan/misc.py:399: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

INFO:fbprophet:n_changepoints greater than number of observations.Using 17.
INFO:fbprophet:n_changepoints greater than number of observations.Using 10.
INFO:fbprophet:n_changepoints greater than number of observations.Using 22.
INFO:fbprophet:n_changepoints greater than number of observations.Using 6.
INFO:fbprophet:n_changepoints greater than number of observat

### I am aggregating  actual and predicted values by Geohash so that it is easy to visualize however  final RMSE is actually average of Geohash

In [16]:
fs_geo_train = pd.merge(fsct_train_mdl,train, how='inner',left_on=['ds','geohash'],right_on=['Datetime','geohash6'])
fs_geo_test = pd.merge(fsct_test_mdl,test, how='left',left_on=['ds','geohash'],right_on=['Datetime','geohash6'])


fs_geo_test = fillnan(fs_geo_test ,"demand")

fs_overall_train = fs_geo_train.groupby(['ds']).agg(
    {
         'yhat':sum,    
         'yhat_lower': sum,  
         'yhat_upper': sum,
         'demand': sum  
    }).reset_index()

fs_overall_test = fs_geo_test.groupby(['ds']).agg(
    {
         'yhat':sum,    
         'yhat_lower': sum,  
         'yhat_upper': sum,
         'demand': sum  
    }).reset_index()

# Replace negative values with 0
fs_overall_train = replacenegative(fs_overall_train)
fs_overall_test = replacenegative(fs_overall_test)

## Forecast Train Visualization

In [17]:
layout = dict(title = 'Forecasting Train',
              xaxis = dict(title = 'Day'),
              yaxis = dict(title = 'Demand/Forecasting'),
              )

demand = plotlyline(fs_overall_train["ds"],fs_overall_train["demand"],"lines","demand")
yhat = plotlyline(fs_overall_train["ds"],fs_overall_train["yhat"],"lines","yhat")
yhat_lower = plotlyline(fs_overall_train["ds"],fs_overall_train["yhat_lower"],"lines","yhat_lower")
yhat_upper = plotlyline(fs_overall_train["ds"],fs_overall_train["yhat_upper"],"lines","yhat_upper")
data = [demand, yhat, yhat_lower, yhat_upper]


fig = dict(data = data, layout=layout)
py.iplot(fig, filename='Forecast plot')

In [18]:
layout = dict(title = 'Forecasting Train',
              xaxis = dict(title = 'Day'),
              yaxis = dict(title = 'Demand/Forecasting'),
              )

demand = plotlyline(fs_overall_train["ds"],fs_overall_train["demand"],"lines","demand")
yhat = plotlyline(fs_overall_train["ds"],fs_overall_train["yhat"],"lines","yhat")
data = [demand, yhat]


fig = dict(data = data, layout=layout)
py.iplot(fig, filename='Forecast plot')

   ### Forecast Test Visualization

In [19]:
layout = dict(title = 'Forecasting Test',
              xaxis = dict(title = 'Day'),
              yaxis = dict(title = 'Demand/Forecasting'),
              )

demand = plotlyline(fs_overall_test["ds"],fs_overall_test["demand"],"lines","demand")
yhat = plotlyline(fs_overall_test["ds"],fs_overall_test["yhat"],"lines","yhat")
yhat_lower = plotlyline(fs_overall_test["ds"],fs_overall_test["yhat_lower"],"lines","yhat_lower")
yhat_upper = plotlyline(fs_overall_test["ds"],fs_overall_test["yhat_upper"],"lines","yhat_upper")
data = [demand, yhat, yhat_lower, yhat_upper]


fig = dict(data = data, layout=layout)
py.iplot(fig, filename='Forecast plot')

In [20]:
layout = dict(title = 'Forecasting Test',
              xaxis = dict(title = 'Day'),
              yaxis = dict(title = 'Demand/Forecasting'),
              )

demand = plotlyline(fs_overall_test["ds"],fs_overall_test["demand"],"lines","demand")
yhat = plotlyline(fs_overall_test["ds"],fs_overall_test["yhat"],"lines","yhat")
data = [demand, yhat]


fig = dict(data = data, layout=layout)
py.iplot(fig, filename='Forecast plot')

### Test dataset which was not used to train model showed RMSE of 0.06 compared to 0.05 in train dataset


### RMSE (root mean squared error) averaged over all geohash6, 15-minute-bucket pairs.

### If we know which city then we can add On/Off season, Holiday calendar, Weather to improve accuracy

In [21]:
print("Overall RMSE of Train",rmse(fs_geo_train["demand"],fs_geo_train["yhat"]))
print("Overall RMSE of Test",rmse(fs_geo_test["demand"],fs_geo_test["yhat"]))

Overall RMSE of Train 0.04988658578580629
Overall RMSE of Test 0.059597760594303786
